In [13]:
import requests
import json
import time
import datetime
import pandas as pd

# 请求评论api接口
def requestApi(url):
    headers = {
        'accept': '*/*',
        'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1',
    }
    try:
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        return r.text
    except requests.HTTPError as e:
        print(e)
    except requests.RequestException as e:
        print(e)
    except:
        print("获取数据出错")
    return None  # 异常时返回None，方便后续判断

# 解析接口返回数据
def getData(html):
    if not html:
        return []
    try:
        json_data = json.loads(html)['cmts']
        comments = []
        for item in json_data:
            comment = {
                'nickName': item['nickName'],
                'cityName': item.get('cityName', ''),  # 无该字段时返回空字符串
                'content': item['content'].strip().replace('\n', ''),
                'score': item['score'],
                'startTime': item['startTime']
            }
            comments.append(comment)
        return comments
    except Exception as e:
        print(f"解析数据出错: {e}")
        return []

# 保存数据，写入CSV（带表头）
def saveData(comments, is_first):
    if not comments:
        return
    filename = './comments_new.csv'
    data_df = pd.DataFrame(comments)
    # 根据是否是首次写入，决定是否添加表头
    data_df.to_csv(
        filename,
        mode='a',
        encoding="utf_8_sig",
        index=False,
        sep=',',
        header=is_first
    )

# 爬虫主函数
def main():
    start = datetime.datetime.now()
    # 定义时间范围（字符串格式）
    start_time_str = '2023-03-06  00:00:00'
    end_time_str = '2022-01-01  00:00:00'
    # 转换为datetime对象，用于比较
    start_time = datetime.datetime.strptime(start_time_str, '%Y-%m-%d  %H:%M:%S')
    end_time = datetime.datetime.strptime(end_time_str, '%Y-%m-%d  %H:%M:%S')

    print(f'开始获取数据! 时间范围: {start_time_str} 至 {end_time_str}')
    is_first_write = True  # 标记是否为首次写入（控制表头）
    page_count = 0  # 记录页数，方便查看进度

    while start_time > end_time:
        page_count += 1
        # 将datetime对象转回字符串，用于拼接URL
        start_time_url = start_time.strftime('%Y-%m-%d  %H:%M:%S').replace('  ', '%20')
        url = f'http://m.maoyan.com/mmdb/comments/movie/248906.json?_v_=yes&offset=0&startTime={start_time_url}'
        print(f'第 {page_count} 页，请求URL: {url}')

        html = requestApi(url)
        if not html:
            time.sleep(2)
            continue

        comments = getData(html)
        if not comments:
            print('本页无有效评论，尝试下一页')
            # 手动更新start_time（避免死循环）
            start_time = start_time + datetime.timedelta(days=-1)
            time.sleep(1)
            continue

        # 保存数据（首次写入时添加表头）
        saveData(comments, is_first_write)
        is_first_write = False  # 首次写入后，后续都不添加表头

        # 更新下一次请求的起始时间（取最后一条评论的时间并减1秒）
        last_comment_time_str = comments[-1]['startTime']
        last_comment_time = datetime.datetime.strptime(last_comment_time_str, '%Y-%m-%d  %H:%M:%S')
        start_time = last_comment_time + datetime.timedelta(seconds=-1)

        print(f'第 {page_count} 页数据已保存，下一次起始时间: {start_time}')
        time.sleep(1)  # 控制爬取速度，避免被反爬

    end = datetime.datetime.now()
    print('获取数据完成！')
    print(f'程序运行用时: {end - start}')
    print(f'共爬取 {page_count} 页数据')

if __name__ == '__main__':
    main()

开始获取数据! 时间范围: 2023-03-06  00:00:00 至 2022-01-01  00:00:00
第 1 页，请求URL: http://m.maoyan.com/mmdb/comments/movie/248906.json?_v_=yes&offset=0&startTime=2023-03-06%2000:00:00
第 1 页数据已保存，下一次起始时间: 2025-09-01 15:28:16
第 2 页，请求URL: http://m.maoyan.com/mmdb/comments/movie/248906.json?_v_=yes&offset=0&startTime=2025-09-01%2015:28:16
第 2 页数据已保存，下一次起始时间: 2025-09-01 15:28:16
第 3 页，请求URL: http://m.maoyan.com/mmdb/comments/movie/248906.json?_v_=yes&offset=0&startTime=2025-09-01%2015:28:16
第 3 页数据已保存，下一次起始时间: 2025-09-01 15:28:16
第 4 页，请求URL: http://m.maoyan.com/mmdb/comments/movie/248906.json?_v_=yes&offset=0&startTime=2025-09-01%2015:28:16
第 4 页数据已保存，下一次起始时间: 2025-09-01 15:28:16
第 5 页，请求URL: http://m.maoyan.com/mmdb/comments/movie/248906.json?_v_=yes&offset=0&startTime=2025-09-01%2015:28:16
第 5 页数据已保存，下一次起始时间: 2025-09-01 15:28:16
第 6 页，请求URL: http://m.maoyan.com/mmdb/comments/movie/248906.json?_v_=yes&offset=0&startTime=2025-09-01%2015:28:16
第 6 页数据已保存，下一次起始时间: 2025-09-01 15:28:16
第 7 页，请求URL: http:

KeyboardInterrupt: 

In [14]:
import pandas as pd
data = pd.read_csv('./comments_new.csv')
data

,nickName,cityName,content,score,startTime
0,小王子5294,马鞍山,剧情可以，推荐看,4.5,2025-09-04 11:43:24
1,紫心168,重庆,非常好看🤩,5.0,2025-09-03 21:24:15
2,蛋黄NAN瓜,上海,中国也有科幻电影了,5.0,2025-09-03 20:50:09
3,XgL408408691,聊城,剧情很好，没有尿点,4.5,2025-09-03 20:15:46
4,重剑无锋,上海,好看 国产科幻天花板 再有续集还要看,5.0,2025-09-03 15:09:49
...,...,...,...,...,...
160,漂流彝族,天津,1. 全程无尿点！剧情紧凑到让人攥紧手心，伏笔埋得巧妙，结尾反转更是让人眼前一亮，走出影院还...,5.0,2025-09-02 10:15:27
161,YJW开心小吃货,毕节,第一次看电影睡着了,2.5,2025-09-02 08:39:42
162,wGc345058377,张家口,剧情好，画面好，演技好,5.0,2025-09-02 05:44:36
163,飘零为新生,南宁,很不错，推荐,4.5,2025-09-01 20:22:22


In [15]:
import pandas as pd
import numpy as np
#!pip3 install jieba
import jieba

data = pd.read_csv('./comments_new.csv').astype(str)
# , names=['Name', 'Area', 'comment', 'star', 'time'])
data['score'] = data['score'].replace(regex=True, inplace=False, to_replace=['nan'], value='')
data1 = data[~data['score'].isin(['0', ' '])]
data1['score'] = pd.to_numeric(data1['score'], errors='coerce')
data1['score'] = data1['score'].apply(
    lambda x: '1' if x in [0.5,1,1.5,2]  # 1、2 → 低
              else '2' if x in [2.5,3,3.5]  # 3 → 中
              else '3'             # 4、5 → 高
)
# data1['score'] = data1['score'].map(type_dict)
data1['cut'] = data1["content"].apply(lambda x: ' '.join(jieba.cut(x)))

def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

# 将短评中的停用词删去
def sentence_div(text):
    # 将短评按空格划分成单词并形成列表
    sentence = text.strip().split()
    # 加载停用词的路径
    stopwords = stopwordslist(r'cn-stopwords.txt')
    # 创建一个空字符串
    outstr = ''
    # 遍历短评列表中每个单词
    for word in sentence:
        if word not in stopwords:  # 判断词汇是否在停用词表里
            if len(word) >= 1:  # 单词长度要大于1
                if word != '\t':  # 单词不能为tab
                    if word not in outstr:  # 去重: 如果单词在outstr中则不加入
                        outstr += ' '  # 分割
                        outstr += word  # 将词汇加入outstr
    # 返回字符串
    return outstr

data1['content_cut'] = data1['cut'].apply(sentence_div)
data1 = data1[['content', 'score', 'content_cut']]
data1
# data1.to_csv("./all_comments.csv", index=None)

,content,score,content_cut
0,剧情可以，推荐看,3,剧情 推荐
1,非常好看🤩,3,好看 🤩
2,中国也有科幻电影了,3,中国 科幻电影
3,剧情很好，没有尿点,3,剧情 尿点
4,好看 国产科幻天花板 再有续集还要看,3,好看 国产 科幻 天花板 续集
...,...,...,...
160,1. 全程无尿点！剧情紧凑到让人攥紧手心，伏笔埋得巧妙，结尾反转更是让人眼前一亮，走出影院还...,3,全程无 尿点 剧情 紧凑 攥 手心 伏笔 埋 巧妙 结尾 反转 更是 眼前一亮 走出 影院...
161,第一次看电影睡着了,2,第一次 电影 睡着
162,剧情好，画面好，演技好,3,剧情 画面 演技
163,很不错，推荐,3,不错 推荐
